In [1]:
import os
import time
import numpy as np
import wurlitzer

import trtlab
import infer_test_utils as utils

# this allows us to capture stdout and stderr from the backend c++ infer-runtime
display_output = wurlitzer.sys_pipes

In [2]:
inputs = utils.load_inputs("/work/models/onnx/mnist-v1.3/test_data_set_0")
expected = utils.load_outputs("/work/models/onnx/mnist-v1.3/test_data_set_0")

In [3]:
!trtexec --onnx=/work/models/onnx/mnist-v1.3/model.onnx --saveEngine=/tmp/mnist-v1.3.engine

onnx: /work/models/onnx/mnist-v1.3/model.onnx
Unknown argument: --saveEngine=/tmp/mnist-v1.3.engine


# Local Inference Setup

In [4]:
with display_output():
    manager = trtlab.InferenceManager(max_exec_concurrency=2)

I0302 00:16:57.011472    52 inference_manager.cc:64] -- Initialzing TensorRT Resource Manager --
I0302 00:16:57.011492    52 inference_manager.cc:65] Maximum Execution Concurrency: 2
I0302 00:16:57.011497    52 inference_manager.cc:66] Maximum Copy Concurrency: 4


In [ ]:
with display_output():
    manager.register_tensorrt_engine("mnist", "/tmp/mnist-v1.3.engine")

In [ ]:
with display_output():
    manager.update_resources()

# Local Inference Properties

In [ ]:
mnist = manager.infer_runner("mnist")

In [ ]:
mnist.input_bindings()

In [ ]:
mnist.output_bindings()

# Local Inference Compute

In [ ]:
futures = [mnist.infer(Input3=input) for input in inputs]

In [ ]:
# free to do other work while inference is being computed

In [ ]:
results = [f.get() for f in futures]

In [ ]:
for r, e in zip(results, expected):
    for key, val in r.items():
        r = val.reshape((1,10))
        np.testing.assert_almost_equal(r, e, decimal=3)
        print("Test Passed")
        print("Result: {}".format(np.argmax(utils.softmax(r))))

utils.mnist_image(inputs[0]).show()
expected[0]